In [1]:
import pyalps
import numpy as np
import matplotlib.pyplot as plt
import pyalps.plot
import pandas as pd

In [2]:
betas = 1. / np.linspace(0.7, 1.4, 29)
1. / betas

array([0.7  , 0.725, 0.75 , 0.775, 0.8  , 0.825, 0.85 , 0.875, 0.9  ,
       0.925, 0.95 , 0.975, 1.   , 1.025, 1.05 , 1.075, 1.1  , 1.125,
       1.15 , 1.175, 1.2  , 1.225, 1.25 , 1.275, 1.3  , 1.325, 1.35 ,
       1.375, 1.4  ])

In [3]:
#prepare the input parameters
parms = []
for beta in betas:
    parms.append(
    {
          'LATTICE'        : "simple cubic lattice", 
          'MODEL'          : "spin",
          'local_S'        : 0.5,
          'L'              : 10,
          'J'              : 1 ,
          'BETA'           : beta,
          'THERMALIZATION' : 10000,
          'SWEEPS'         : 10000,
    }
)


In [4]:
input_file = pyalps.writeInputFiles('3D-SSE-Heisenberg', parms)
res = pyalps.runApplication('dirloop_sse', input_file)

dirloop_sse 3D-SSE-Heisenberg.in.xml


In [5]:
data = pyalps.loadMeasurements(pyalps.getResultFiles(prefix='3D-SSE-Heisenberg'))

- |Magnetization Density|
- Staggered Magnetization Density^2
- Magnetization^2
- Susceptibility
- n
- n^2
- Staggered Magnetization^4
- Magnetization Density^4
- Staggered Magnetization Density^4
- Magnetization Density^2
- Stiffness
- n^3
- Staggered Magnetization
- Staggered Magnetization Density
- Energy
- Staggered Magnetization^2
- Kinetic Energy Density
- Kinetic Energy
- Magnetization^4
- |Magnetization|
- Magnetization Density
- Energy Density
- Magnetization

In [8]:
observables = ['Energy', 'n', 'n^2', 'Susceptibility', 'Staggered Magnetization']

In [9]:
for obs in observables:
    res = []
    for aux in data:
        for aux2 in aux:
            if (aux2.props['observable'] == obs):
                
                split = str(aux2.y[0]).split('+/-')

                val, err = split[0], split[1]

                val = float(val) / aux2.props['L']**3
                err =  float(err) / aux2.props['L']**3

                beta = float(aux2.props['BETA'])
                
                res.append({'beta': beta, obs: val, obs + '_err': err})
                
    pd.DataFrame(res).sort_values('beta', ascending=False).to_csv('ALPS_' + obs + '.dat', index=False, sep=' ')